In [ ]:
# create env
import hydra
from calvin_agent.evaluation.multistep_sequences import get_sequences
from calvin_agent.evaluation.utils import get_env_state_for_initial_condition
from hydra import compose, initialize
from lightning_lite import seed_everything

from manten_evaluation.calvin.evaluate_policy import make_env

with initialize(config_path="../manten_evaluation/calvin/configs"):
    cfg = compose(config_name="evaluate_policy")

seed_everything(0, workers=True)

env = make_env(cfg.dataset_path)

In [ ]:
import pyhash
from optree import tree_map

hasher = pyhash.fnv1_64()
teha = lambda y: tree_map(lambda x: hasher(str(x.tolist() if hasattr(x, "tolist") else x)), y)  # noqa: E731

In [ ]:
import torch

initial_state, eval_sequence = get_sequences(1)[0]

robot_obs, scene_obs = get_env_state_for_initial_condition(initial_state)
env.reset(robot_obs=robot_obs, scene_obs=scene_obs)

obs_raw = env.get_obs()

start_info = env.get_info()

display(hasher(str(start_info)))
# display(teha(obs_raw))
display(hasher(str(teha(obs_raw))))

obs = None
goal = None


class MockClient:
    def __init__(self):
        pass

    def reset(self):
        pass

    def step(self, tobs, tgoal):
        global obs  # noqa: PLW0603
        global goal  # noqa: PLW0603
        obs = tobs
        goal = tgoal
        return torch.rand(7)


cfg.agent_proxy.client = None
model = hydra.utils.instantiate(cfg.agent_proxy)
model.client = MockClient()

act = model.step(obs_raw, "langinfo")
display(hasher(str(goal)))
display(teha(obs))